In [1]:
import pandas as pd

In [2]:
file_path = '/Users/vree/Documents/UTwente/Masters UTwente/Data Science/DM Project/Speed-flow/Mei/intensiteit-snelheid-rotterdam-1 mei_metadata/intensiteit-snelheid-rotterdam-1 mei metadata.csv'
# Define the columns to load
cols_to_load = ['id_meetlocatie', 'start_locatie_longitude', 'start_locatie_latitude', 'wegnaam_vild', 'rijstrook_rijbaan']
cols_for_uniqueness = ['id_meetlocatie', 'start_locatie_longitude', 'start_locatie_latitude']
# Initialize an empty DataFrame to hold unique rows
unique_rows = pd.DataFrame(columns=cols_to_load)

# Set chunk size
chunk_size = 20000

# Read the CSV file in chunks
for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_size):
    # Drop duplicates within the chunk
    chunk_unique = chunk.drop_duplicates(subset=cols_to_load)
    
    # Concatenate the unique rows from this chunk with the main DataFrame, and drop duplicates again
    unique_rows = pd.concat([unique_rows, chunk_unique]).drop_duplicates(subset=cols_for_uniqueness)

unique_rows = unique_rows[unique_rows['wegnaam_vild'] == 'Ring Rotterdam']
# Print the size of the reduced DataFrame
print(unique_rows.shape)

/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_3523/960211014.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unique_rows = pd.concat([unique_rows, chunk_unique]).drop_duplicates(subset=cols_for_uniqueness)
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_3523/960211014.py:12: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_size):
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_3523/960211014.py:12: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_siz

(635, 5)


In [3]:
unique_rows.head(5)

,id_meetlocatie,start_locatie_longitude,start_locatie_latitude,wegnaam_vild,rijstrook_rijbaan
0,GEO0B_R_RWSTI356574,4.368670,51.910990,Ring Rotterdam,lane1
103608,GEO0B_R_RWSTI357193,4.370783,51.910187,Ring Rotterdam,lane1
120876,GEO0B_R_RWSTI357194,4.370130,51.911260,Ring Rotterdam,lane1
129510,GEO0B_R_RWSTI357196,4.373160,51.926146,Ring Rotterdam,lane1
138144,GEO0B_R_RWSTI357197,4.374173,51.926116,Ring Rotterdam,lane1


In [4]:
import folium

# rename uniquerows id_meetlocatie to id
unique_rows.rename(columns={'id_meetlocatie': 'id'}, inplace=True)

In [5]:
# Create a map centered on the mean latitude and longitude
m = folium.Map(location=[unique_rows['start_locatie_latitude'].mean(), unique_rows['start_locatie_longitude'].mean()], zoom_start=10)

# Add a marker for each unique meetlocatie

for index, row in unique_rows.iterrows():
    folium.Marker(
        [row['start_locatie_latitude'], row['start_locatie_longitude']],
        popup=f"ID: {row['id']}<br>start_locatie_latitude: {row['start_locatie_latitude']:.6f}<br>start_locatie_longitude: {row['start_locatie_longitude']:.6f}",
        tooltip='Click for more info'
    ).add_to(m)

# Show the map
m.save('clickable_ring_cameras.html')
m